In [1]:
import torch
torch.cuda.is_available()

True

In [4]:
import numpy as np
from sklearn.metrics import roc_auc_score, roc_curve, average_precision_score
from sklearn.metrics import precision_recall_curve, auc
from scipy.spatial import distance

def predict(l,emb):
    r = []
    p = []
    for p1,p2,real in l:
        dist = abs(emb[p1]-emb[p2]).sum()/2
        r.append(real)
        p.append(-dist)
    return p,r

def predictAll(dbname,emb):
    if dbname == 'pfam': l1 = phoml; l2 = pnonl
    elif dbname == 'gene3d': l1 = ghoml; l2 = gnonl
    else: l1 = shoml; l2 = snonl
    ph,rh = predict(l1,emb)
    pn,rn = predict(l2,emb)
    p = ph+pn
    r = rh+rn
    return p,r

def aucNth(y, yp, N):
    assert len(y) == len(yp)
    assert len(y) > 1
    fpr, tpr, thresholds = roc_curve(y, yp)
    negatives = y.count(0)
    assert N < negatives
    perc = N / float(negatives)
    fpr1k = []
    tpr1k = []
    i = 0
    while i < len(fpr):
        if fpr[i] > perc:
            break
        fpr1k.append(fpr[i])
        tpr1k.append(tpr[i])
        i+=1
    assert len(fpr1k) > 1
    aucScore = auc(fpr1k, tpr1k) / perc
    return aucScore

def score(dbname,emb,aucN=1000):
    p,r = predictAll(dbname,emb)
    print(len(p),len(r),len(phoml),len(pnonl))
    print(p[:10])
    fpr, tpr, threshold = roc_curve(r, p)
    aucp = aucNth(r, p, aucN)
    aucScore = roc_auc_score(r,p)
    print(dbname,aucScore,aucp)
    return fpr,tpr,aucScore,aucp


In [2]:
from tools import read_benchmark_pairs, embed_seq, read_fasta, dump_pickle, load_pickle
import os

phom = '../benchmark/Methods_benchmarking_pairs/pfam_hom-pairs_max50.txt'
pnon = '../benchmark/Methods_benchmarking_pairs/pfam_nonhom-pairs_max50.txt'
ghom = '../benchmark/Methods_benchmarking_pairs/gene3d_hom-pairs_max50.txt'
gnon = '../benchmark/Methods_benchmarking_pairs/gene3d_nonhom-pairs_max50.txt'
shom = '../benchmark/Methods_benchmarking_pairs/supfam_hom-pairs_max50.txt'
snon = '../benchmark/Methods_benchmarking_pairs/supfam_nonhom-pairs_max50.txt'

phoml = read_benchmark_pairs(phom,1)
pnonl = read_benchmark_pairs(pnon,0)
ghoml = read_benchmark_pairs(ghom,1)
gnonl = read_benchmark_pairs(gnon,0)
shoml = read_benchmark_pairs(shom,1)
snonl = read_benchmark_pairs(snon,0)

max50seq = read_fasta('../benchmark/max50.fa')

if os.path.exists('max50emb.pkl') and os.path.exists('max50log.pkl'):
    print('Loading...')
    max50emb = load_pickle('max50emb.pkl')
    max50log = load_pickle('max50log.pkl')
else:
    max50emb,max50log = embed_seq(max50seq)
    print('Dumping...')
    dump_pickle('max50emb.pkl',max50emb)
    dump_pickle('max50log.pkl',max50log)

Loading...


In [23]:
fpr,tpr,aucScore,aucp = score('pfam',max50emb)
dump_pickle('pfam.res.pkl',(fpr,tpr,aucScore,aucp))

fpr,tpr,aucScore,aucp = score('supfam',max50emb)
dump_pickle('supfam.res.pkl',(fpr,tpr,aucScore,aucp))

fpr,tpr,aucScore,aucp = score('gene3d',max50emb)
dump_pickle('gene3d.res.pkl',(fpr,tpr,aucScore,aucp))

pfam 0.9933850478143877 0.9590841754051477
supfam 0.9902220887085216 0.9662332078462452
gene3d 0.9894486885039282 0.9512830622347949


In [19]:
from tools import read_benchmark_pairs, embed_seq, read_fasta, dump_pickle, load_pickle
import os

phom = '../benchmark/Methods_benchmarking_pairs/pfam_hom-pairs_nomax50.txt'
pnon = '../benchmark/Methods_benchmarking_pairs/pfam_nonhom-pairs_nomax50.txt'
ghom = '../benchmark/Methods_benchmarking_pairs/gene3d_hom-pairs_nomax50.txt'
gnon = '../benchmark/Methods_benchmarking_pairs/gene3d_nonhom-pairs_nomax50.txt'
shom = '../benchmark/Methods_benchmarking_pairs/supfam_hom-pairs_nomax50.txt'
snon = '../benchmark/Methods_benchmarking_pairs/supfam_nonhom-pairs_nomax50.txt'

phoml = read_benchmark_pairs(phom,1)
pnonl = read_benchmark_pairs(pnon,0)
ghoml = read_benchmark_pairs(ghom,1)
gnonl = read_benchmark_pairs(gnon,0)
shoml = read_benchmark_pairs(shom,1)
snonl = read_benchmark_pairs(snon,0)

nomax50seq = read_fasta('../benchmark/nomax.fa')

if os.path.exists('nomax50emb.pkl') and os.path.exists('nomax50log.pkl'):
    print('Loading...')
    nomax50emb = load_pickle('nomax50emb.pkl')
    nomax50log = load_pickle('nomax50log.pkl')
else:
    nomax50emb,nomax50log = embed_seq(nomax50seq)
    print('Dumping...')
    dump_pickle('nomax50emb.pkl',nomax50emb)
    dump_pickle('nomax50log.pkl',nomax50log)

Loading...


In [20]:
fpr,tpr,aucScore,aucp = score('pfam',nomax50emb,10000)
dump_pickle('pfam.no50.res.pkl',(fpr,tpr,aucScore,aucp))

fpr,tpr,aucScore,aucp = score('supfam',nomax50emb,10000)
dump_pickle('supfam.no50.res.pkl',(fpr,tpr,aucScore,aucp))

fpr,tpr,aucScore,aucp = score('gene3d',nomax50emb,10000)
dump_pickle('gene3d.no50.res.pkl',(fpr,tpr,aucScore,aucp))